# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#Valuation-Model"><span class="toc-item-num">3 - </span>Valuation Model</a></div>

# Import `Python` modules

In [1]:
from __future__ import print_function
import numpy
import pandas
from pprint import pprint

# Set _CONSTANTS_

In [2]:
# Revenue Y-o-Y Growth Rates From 2005 to 2015
REV_GROWTH_RATES_2005_2015 = [.5, .4, .3, .2, .2, .2, .2, .2, .2, .2, .05]

# EBIAT / Revenue Ratios from 2005 to 2015
EBIAT_OVER_REVENUE_RATIOS_2005_2015 = [.04, .05, .06, .06, .06, .06, .06, .06, .06, .06, .06]

# Further Investments & Working Capital Increases / Revenue Ratio
FIXED_ASSETS_AND_WORKING_CAP_OVER_REVENUE_RATIO = -.5

# DISCOUNT RATE
DISCOUNT_RATE = .174

# Long-Term Growth Rate
LONG_TERM_GROWTH_RATE = .04

# Number of Pro-Forma Years EXCLUDING Year 0
NB_PRO_FORMA_YEARS = 12

# Valuation Model

In [3]:
# Valuation Calculations data frame
val_calcs_df = pandas.DataFrame(index=range(2003, 2016))
pandas.options.display.float_format = '{:,.2f}'.format

val_calcs_df.T

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015


Revenue projection from assumed growth rates:

In [4]:
Rev = [29.3, 60]

for g in REV_GROWTH_RATES_2005_2015:
    Rev.append((1 + g) * Rev[-1])

val_calcs_df['Revenue'] = Rev

val_calcs_df.T

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenue,29.30,60.00,90.00,126.00,163.80,196.56,235.87,283.05,339.66,407.59,489.10,586.93,616.27


EBIAT projection according to assumed EBIAT / Revenue ratios:

In [5]:
val_calcs_df['EBIAT / Revenue'] = [0, 0] + EBIAT_OVER_REVENUE_RATIOS_2005_2015

val_calcs_df['EBIAT'] = val_calcs_df['EBIAT / Revenue'] * val_calcs_df.Revenue

val_calcs_df.T

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenue,29.30,60.00,90.00,126.00,163.80,196.56,235.87,283.05,339.66,407.59,489.10,586.93,616.27
EBIAT / Revenue,0.00,0.00,0.04,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
EBIAT,0.00,0.00,3.60,6.30,9.83,11.79,14.15,16.98,20.38,24.46,29.35,35.22,36.98


Investment and Working Capital Change projection according to ratios on Revenue:

In [6]:
val_calcs_df['Fixed Assets & Working Capital Balance'] = \
    FIXED_ASSETS_AND_WORKING_CAP_OVER_REVENUE_RATIO * val_calcs_df.Revenue
    
val_calcs_df['Fixed Assets & Working Capital Change'] = \
    val_calcs_df['Fixed Assets & Working Capital Balance'] - \
    val_calcs_df['Fixed Assets & Working Capital Balance'].shift()

val_calcs_df.T

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenue,29.30,60.00,90.00,126.00,163.80,196.56,235.87,283.05,339.66,407.59,489.10,586.93,616.27
EBIAT / Revenue,0.00,0.00,0.04,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
EBIAT,0.00,0.00,3.60,6.30,9.83,11.79,14.15,16.98,20.38,24.46,29.35,35.22,36.98
Fixed Assets & Working Capital Balance,-14.65,-30.00,-45.00,-63.00,-81.90,-98.28,-117.94,-141.52,-169.83,-203.79,-244.55,-293.46,-308.14
Fixed Assets & Working Capital Change,nan,-15.35,-15.00,-18.00,-18.90,-16.38,-19.66,-23.59,-28.30,-33.97,-40.76,-48.91,-14.67


Free Cash Flows:

In [7]:
val_calcs_df['FCF before TV'] = \
    val_calcs_df.EBIAT - val_calcs_df['Fixed Assets & Working Capital Change']
    
val_calcs_df.T

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenue,29.30,60.00,90.00,126.00,163.80,196.56,235.87,283.05,339.66,407.59,489.10,586.93,616.27
EBIAT / Revenue,0.00,0.00,0.04,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
EBIAT,0.00,0.00,3.60,6.30,9.83,11.79,14.15,16.98,20.38,24.46,29.35,35.22,36.98
Fixed Assets & Working Capital Balance,-14.65,-30.00,-45.00,-63.00,-81.90,-98.28,-117.94,-141.52,-169.83,-203.79,-244.55,-293.46,-308.14
Fixed Assets & Working Capital Change,nan,-15.35,-15.00,-18.00,-18.90,-16.38,-19.66,-23.59,-28.30,-33.97,-40.76,-48.91,-14.67
FCF before TV,nan,15.35,18.60,24.30,28.73,28.17,33.81,40.57,48.68,58.42,70.10,84.13,51.65


Terminal Value:

In [8]:
val_calcs_df['Terminal Value'] = 0
val_calcs_df.loc[2015, 'Terminal Value'] = \
    (1 + LONG_TERM_GROWTH_RATE) * val_calcs_df.loc[2015, 'FCF before TV'] / \
    (DISCOUNT_RATE - LONG_TERM_GROWTH_RATE)
    
val_calcs_df['FCF + TV'] = \
    val_calcs_df['FCF before TV'] + val_calcs_df['Terminal Value']
    
val_calcs_df.T

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenue,29.30,60.00,90.00,126.00,163.80,196.56,235.87,283.05,339.66,407.59,489.10,586.93,616.27
EBIAT / Revenue,0.00,0.00,0.04,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
EBIAT,0.00,0.00,3.60,6.30,9.83,11.79,14.15,16.98,20.38,24.46,29.35,35.22,36.98
Fixed Assets & Working Capital Balance,-14.65,-30.00,-45.00,-63.00,-81.90,-98.28,-117.94,-141.52,-169.83,-203.79,-244.55,-293.46,-308.14
Fixed Assets & Working Capital Change,nan,-15.35,-15.00,-18.00,-18.90,-16.38,-19.66,-23.59,-28.30,-33.97,-40.76,-48.91,-14.67
FCF before TV,nan,15.35,18.60,24.30,28.73,28.17,33.81,40.57,48.68,58.42,70.10,84.13,51.65
Terminal Value,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,400.86
FCF + TV,nan,15.35,18.60,24.30,28.73,28.17,33.81,40.57,48.68,58.42,70.10,84.13,452.51


Valuation:

In [9]:
val = numpy.npv(
    rate=DISCOUNT_RATE,
    values=val_calcs_df.loc[range(2004, 2016), 'FCF + TV']) / (1 + DISCOUNT_RATE)

print('Valuation = $%.2f MM' % val)

Valuation = $217.25 MM
